# Evaluating AI Models: Code, Human, and Model-Based Grading

（AIモデルの評価：コード採点・人手採点・モデル採点）

このノートブックでは、Claude v3 のような AI モデルの有効性を評価するために広く使われている 3 つの手法を扱います。

1. コードによる採点（Code-based grading）
2. 人手による採点（Human grading）
3. モデルによる採点（Model-based grading）

それぞれを例で示し、AI の性能評価における利点と限界を確認します。

## Code-Based Grading Example: Sentiment Analysis

この例では、映画レビューの感情（ポジティブ／ネガティブ）を分類する Claude の能力を評価します。モデルの出力が期待されるラベルと一致しているかを、コードでチェックできます。

In [1]:
# Store the model name and AWS region for later use
MODEL_NAME = "anthropic.claude-3-haiku-20240307-v1:0"
AWS_REGION = "us-west-2"

# %store MODEL_NAME
# %store AWS_REGION

In [36]:
%store -r MODEL_NAME
MODEL_NAME

'us.anthropic.claude-sonnet-4-20250514-v1:0'

In [ ]:
# Install the Anthropic package
# %pip install anthropic --quiet

In [2]:
# Import the AnthropicBedrock class and create a client instance
from anthropic import AnthropicBedrock

client = AnthropicBedrock(aws_region=AWS_REGION)

In [38]:
# Function to build the input prompt for sentiment analysis
def build_input_prompt(review):
    user_content = f"""Classify the sentiment of the following movie review as either 'positive' or 'negative' provide only one of those two choices:
    <review>{review}</review>"""
    return [{"role": "user", "content": user_content}]

# Define the evaluation data
eval = [
    {
        "review": "This movie was amazing! The acting was superb and the plot kept me engaged from start to finish.",
        "golden_answer": "positive"
    },
    {
        "review": "I was thoroughly disappointed by this film. The pacing was slow and the characters were one-dimensional.",
        "golden_answer": "negative"
    }
]

# Function to get completions from the model
def get_completion(messages, system_prompt="日本語で回答してください"):
    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=messages,
        system=system_prompt,
    )
    return message.content[0].text
# def get_completion(messages, system_prompt="", prefill=""):
#     message = ''
#     stream = client.messages.create(
#         model=MODEL_NAME,
#         max_tokens=2000,
#         temperature=0.0,
#         messages=messages,
#         system=system_prompt,
#         stream=True
#     )
#     for event in stream:
#         if event.type == "content_block_delta":
#             print(event.delta.text, flush=True, end="")
#             message += event.delta.text
#     return message
    
# Get completions for each input
outputs = [get_completion(build_input_prompt(item["review"])) for item in eval]

# Print the outputs and golden answers
for output, question in zip(outputs, eval):
    print(f"Review: {question['review']}\nGolden Answer: {question['golden_answer']}\nOutput: {output}\n")

# Function to grade the completions
def grade_completion(output, golden_answer):
    return output.lower() == golden_answer.lower()

# Grade the completions and print the accuracy
grades = [grade_completion(output, item["golden_answer"]) for output, item in zip(outputs, eval)]
print(f"Accuracy: {sum(grades) / len(grades) * 100}%")

Review: This movie was amazing! The acting was superb and the plot kept me engaged from start to finish.
Golden Answer: positive
Output: positive

Review: I was thoroughly disappointed by this film. The pacing was slow and the characters were one-dimensional.
Golden Answer: negative
Output: negative

Accuracy: 100.0%


## Human Grading Example: Essay Scoring

エッセイの採点のように、コードだけでは評価しにくいタスクもあります。その場合は、人間の評価者がモデル出力を評価できるように、採点ガイドライン（基準）を用意します。

In [37]:
# Function to build the input prompt for essay generation
def build_input_prompt(topic):
    user_content = f"""Write a short essay discussing the following topic:
    <topic>{topic}</topic>"""
    return [{"role": "user", "content": user_content}]

# Define the evaluation data
eval = [
    {
        "topic": "The importance of education in personal development and societal progress",
        "golden_answer": "A high-scoring essay should have a clear thesis, well-structured paragraphs, and persuasive examples discussing how education contributes to individual growth and broader societal advancement."
    }
]

# Get completions for each input
outputs = [get_completion(build_input_prompt(item["topic"])) for item in eval]

# Print the outputs and golden answers
for output, item in zip(outputs, eval):
    print(f"Topic: {item['topic']}\n\nGrading Rubric:\n {item['golden_answer']}\n\nModel Output:\n{output}\n")

Topic: The importance of education in personal development and societal progress

Grading Rubric:
 A high-scoring essay should have a clear thesis, well-structured paragraphs, and persuasive examples discussing how education contributes to individual growth and broader societal advancement.

Model Output:
# 教育の重要性：個人の成長と社会の発展において

教育は、個人の人格形成と社会全体の進歩において、極めて重要な役割を果たしています。この小論では、教育が個人と社会に与える影響について考察します。

## 個人の発達における教育の役割

教育は、まず個人の知識と技能の習得を可能にします。学校教育を通じて、私たちは読み書き計算といった基礎的な能力から、専門的な知識まで幅広く学びます。しかし、教育の価値はそれだけにとどまりません。

教育は批判的思考力を育成し、問題解決能力を向上させます。様々な情報を分析し、論理的に判断する力は、現代社会を生きる上で不可欠です。また、教育は創造性を刺激し、新しいアイデアや革新的な解決策を生み出す基盤となります。

さらに、教育は人格形成にも大きく寄与します。他者との協働、責任感、倫理観といった社会性を身につけることで、より良い市民として成長することができます。

## 社会の進歩における教育の意義

社会レベルでは、教育は経済発展の原動力となります。高い教育水準を持つ人材は、技術革新を推進し、生産性を向上させ、国際競争力を高めます。知識基盤社会において、教育への投資は最も確実な成長戦略の一つです。

また、教育は社会の平等性を促進します。質の高い教育機会を広く提供することで、出身や経済状況に関係なく、個人の能力と努力に基づいた社会参加が可能になります。これは社会の公正性を高め、格差の縮小に貢献します。

さらに、教育は民主主義の基盤でもあります。情報を適切に理解し、合理的な判断を下せる市民の存在は、健全な民主社会の維持に不可欠です。


## Model-Based Grading Examples

モデルの回答と採点ルーブリック（評価基準）を与えることで、Claude 自身に自分の出力を採点させることもできます。これにより、本来は人間の判断が必要になりがちなタスクの評価を自動化できます。

### Example 1: Summarization

この例では、Claude が生成した要約の品質を Claude 自身に評価させます。長文から重要情報を簡潔かつ正確に抜き出せているかを評価したいときに有用です。含めるべき要点をまとめたルーブリックを与えることで、採点プロセスを自動化し、要約タスクにおけるモデル性能を素早く確認できます。

In [39]:
# Function to build the input prompt for summarization
def build_input_prompt(text):
    user_content = f"""Please summarize the main points of the following text:
    <text>{text}</text>"""
    return [{"role": "user", "content": user_content}]

# Function to build the grader prompt for assessing summary quality
def build_grader_prompt(output, rubric):
    user_content = f"""Assess the quality of the following summary based on this rubric:
    <rubric>{rubric}</rubric>
    <summary>{output}</summary>
    Provide a score from 1-5, where 1 is poor and 5 is excellent."""
    return [{"role": "user", "content": user_content}]

# Define the evaluation data
eval = [
    {
        "text": "The Magna Carta, signed in 1215, was a pivotal document in English history. It limited the powers of the monarchy and established the principle that everyone, including the king, was subject to the law. This laid the foundation for constitutional governance and the rule of law in England and influenced legal systems worldwide.",
        "golden_answer": "A high-quality summary should concisely capture the key points: 1) The Magna Carta's significance in English history, 2) Its role in limiting monarchical power, 3) Establishing the principle of rule of law, and 4) Its influence on legal systems around the world."
    }
]

# Get completions for each input
outputs = [get_completion(build_input_prompt(item["text"])) for item in eval]
print("====== Model Outputs ======")
print(outputs[0])
# Grade the completions
grades = [get_completion(build_grader_prompt(output, item["golden_answer"])) for output, item in zip(outputs, eval)]

# Print the summary quality score
print("====== Grades ======")
print(f"Summary quality score: {grades[0]}")

====== Model Outputs ======
マグナ・カルタに関するテキストの主要なポイントは以下の通りです：

1. **歴史的重要性**: 1215年に署名されたマグナ・カルタは、イングランド史における極めて重要な文書でした。

2. **王権の制限**: この文書は君主の権力を制限する役割を果たしました。

3. **法の下の平等**: 国王を含むすべての人が法に従うという原則を確立しました。

4. **立憲政治の基盤**: イングランドにおける立憲政治と法の支配の基礎を築きました。

5. **世界的影響**: 世界各国の法制度に影響を与えました。

マグナ・カルタは、現代の民主主義と法治国家の概念の源流となった歴史的文書として位置づけられています。
====== Grades ======
Summary quality score: この要約の品質を評価いたします。

**スコア: 4/5 (優秀)**

**評価理由:**

**強み:**
- ルーブリックの4つの重要ポイントをすべて適切にカバーしています
  1. ✓ イングランド史における重要性（1215年の署名、歴史的重要性）
  2. ✓ 君主権力の制限（明確に言及）
  3. ✓ 法の支配の原則（「法の下の平等」「国王を含むすべての人が法に従う」）
  4. ✓ 世界的な法制度への影響（明確に記載）

- 構造が明確で読みやすい番号付きリスト形式
- 簡潔でありながら各ポイントを適切に説明
- 現代への影響も言及し、文書の継続的重要性を示している

**改善点:**
- 「法の下の平等」という表現は、より正確には「法の支配」（rule of law）と表現すべき
- 具体的な影響例（アメリカ憲法、人権宣言など）があればより説得力が増す

全体として、ルーブリックの要求事項を網羅し、簡潔かつ包括的な優秀な要約です。


### Example 2: Fact-Checking

この例では、主張のファクトチェックを Claude に行わせ、そのファクトチェック自体の正確さを評価します。正確な情報と不正確な情報を区別する能力を評価したい場合に有用です。正しいファクトチェックに必要な要点をルーブリックとして与えることで、採点を自動化し、ファクトチェックタスクにおけるモデル性能を素早く確認できます。

In [40]:
# Function to build the input prompt for fact-checking
def build_input_prompt(claim):
    user_content = f"""Determine if the following claim is true or false:
    <claim>{claim}</claim>"""
    return [{"role": "user", "content": user_content}]

# Function to build the grader prompt for assessing fact-check accuracy
def build_grader_prompt(output, rubric):
    user_content = f"""Based on the following rubric, assess whether the fact-check is correct:
    <rubric>{rubric}</rubric>
    <fact-check>{output}</fact-check>"""
    return [{"role": "user", "content": user_content}]

# Define the evaluation data
eval = [
    {
        "claim": "The Great Wall of China is visible from space.",
        "golden_answer": "A correct fact-check should state that this claim is false. While the Great Wall is an impressive structure, it is not visible from space with the naked eye."
    }
]

# Get completions for each input
outputs = [get_completion(build_input_prompt(item["claim"])) for item in eval]

grades = []
for output, item in zip(outputs, eval):
    # Print the claim, fact-check, and rubric
    print(f"Claim: {item['claim']}\n")
    print(f"Fact-check: {output}]\n")
    print(f"Rubric: {item['golden_answer']}\n")
    
    # Grade the fact-check
    print("====== Grades ======")
    grader_prompt = build_grader_prompt(output, item["golden_answer"])
    grade = get_completion(grader_prompt)
    print(grade)
    # grades.append("correct" in grade.lower())
    grades.append("正しい" in grade.lower())

# Print the fact-checking accuracy
accuracy = sum(grades) / len(grades)
print(f"Fact-checking accuracy: {accuracy * 100}%")

Claim: The Great Wall of China is visible from space.

Fact-check: この主張は**偽（False）**です。

「万里の長城が宇宙から見える」という話は広く信じられている都市伝説ですが、実際には正しくありません。

**理由：**

1. **幅の問題**: 万里の長城の幅は約4-9メートルで、宇宙から肉眼で識別するには細すぎます

2. **宇宙飛行士の証言**: 実際に宇宙に行った宇宙飛行士たちは、万里の長城を肉眼では見ることができないと証言しています

3. **NASAの見解**: NASAも公式にこの主張を否定しており、「万里の長城は宇宙から肉眼では見えない」と明言しています

4. **他の人工構造物との比較**: 万里の長城よりもはるかに大きな高速道路や空港でさえ、宇宙からは識別が困難です

ただし、特定の条件下（低軌道、望遠レンズ使用、理想的な照明条件など）では、写真撮影によって確認できる場合があります。しかし、一般的に言われる「肉眼で見える」という主張は事実ではありません。]

Rubric: A correct fact-check should state that this claim is false. While the Great Wall is an impressive structure, it is not visible from space with the naked eye.

====== Grades ======
この事実確認は**正しい**です。

ルーブリックの要求事項との照合：

✅ **主張を偽（False）と判定している**: 事実確認は冒頭で明確に「この主張は**偽（False）**です」と述べています。

✅ **万里の長城が印象的な構造物であることを認めている**: 「広く信じられている都市伝説」という表現で、この主張の普及度と万里の長城の印象的な性質を暗示的に認めています。

✅ **宇宙から肉眼では見えないことを説明している**: 複数の根拠（幅の問題、宇宙飛行士の証言、NASAの見解、他の構造物との比較）を挙げて、肉眼では見えないことを詳しく説明しています。

さらに、この事実確認は以下の点で優れています：
- 科学的根拠に基づ

### Example 3: Tone Analysis

この例では、与えられたテキストのトーン（語調・雰囲気）を Claude に分析させ、その分析の正確さを評価します。文章に含まれる感情的な内容や態度を識別し解釈する能力を評価したい場合に有用です。識別すべきトーンの観点をルーブリックとして与えることで、採点を自動化し、トーン分析タスクにおけるモデル性能を素早く確認できます。

In [41]:
# Function to build the input prompt for tone analysis
def build_input_prompt(text):
    user_content = f"""Analyze the tone of the following text:
    <text>{text}</text>"""
    return [{"role": "user", "content": user_content}]

# Function to build the grader prompt for assessing tone analysis accuracy
def build_grader_prompt(output, rubric):
    user_content = f"""Assess the accuracy of the following tone analysis based on this rubric:
    <rubric>{rubric}</rubric>
    <tone-analysis>{output}</tone-analysis>"""
    return [{"role": "user", "content": user_content}]

# Define the evaluation data
eval = [
    {
        "text": "I can't believe they canceled the event at the last minute. This is completely unacceptable and unprofessional!",
        "golden_answer": "The tone analysis should identify the text as expressing frustration, anger, and disappointment. Key words like 'can't believe', 'last minute', 'unacceptable', and 'unprofessional' indicate strong negative emotions."
    }
]

# Get completions for each input
outputs = [get_completion(build_input_prompt(item["text"])) for item in eval]
print("====== Model Outputs ======")
print(outputs[0])

# Grade the completions
grades = [get_completion(build_grader_prompt(output, item["golden_answer"])) for output, item in zip(outputs, eval)]

# Print the tone analysis quality
print("====== Grades ======")
print(f"Tone analysis quality: {grades[0]}")

====== Model Outputs ======
このテキストの語調を分析すると、以下のような特徴が見られます：

**主な語調：**
- **怒り・憤慨** - 強い不満と怒りが表現されている
- **批判的** - 主催者の対応を厳しく非難している
- **感情的** - 冷静さを欠いた感情的な表現

**具体的な要素：**
- 「I can't believe」- 驚きと信じられないという気持ち
- 「at the last minute」- 直前のキャンセルへの不満
- 「completely unacceptable」- 強い拒絶感
- 「unprofessional」- 相手の対応への厳しい評価
- 感嘆符（!）- 感情の強さを示している

**全体的な印象：**
書き手は非常に不満を抱いており、相手の行動を強く批判している。プロフェッショナルではない対応に対する失望と怒りが明確に表現された、攻撃的で批判的な語調となっています。
====== Grades ======
Tone analysis quality: この語調分析の正確性を評価すると、**非常に高い精度**で要求された基準を満たしています。

## 評価のポイント：

### ✅ **優秀な点：**

1. **感情の正確な特定**
   - ルーブリックで求められた「frustration（苛立ち）」「anger（怒り）」「disappointment（失望）」をすべて適切に識別
   - 「怒り・憤慨」「批判的」「感情的」として的確に表現

2. **キーワードの完全な把握**
   - 指定された4つのキーワード（'can't believe', 'last minute', 'unacceptable', 'unprofessional'）をすべて正確に分析
   - 各語句の感情的な意味合いを適切に解釈

3. **分析の深度**
   - 単なる感情の識別にとどまらず、感嘆符などの文体的要素も考慮
   - 「プロフェッショナルではない対応への失望」など、文脈的な理解も示している

### 📊 **総合評価：A+（95-100%の精度）**

この分析は、ルーブリックの要求事項を完全に満たし、さらに詳細で構造化された洞察を提供しています。語調分析として非常に優秀な品質です

これらの例は、コード採点・人手採点・モデル採点が、Claude のような AI モデルをさまざまなタスクで評価するのに使えることを示しています。どの評価方法を選ぶかは、タスクの性質と利用できるリソースによって決まります。モデル採点は、本来は人間の判断が必要となる複雑なタスク評価を自動化する有望なアプローチです。